In [1]:
import cv2
import os
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

from random import sample
from numpy.linalg import norm
from ultis import *

# Input

In [3]:
image_directory = "datasets/INRIA/images/"
# image_directory = "datasets/UKB/full"

image_paths = get_all_image_paths(image_directory)

In [4]:
# image_paths
len(image_paths) 

1491

# SIFT

In [ ]:
all_features = []
for i, path in enumerate(image_paths):
    key_points, descriptors = extract_sift_features(path)
    
    if descriptors is not None:
        keypoints = sorted(keypoints, key=lambda x: -x.response)
        max_features = 500
        keypoints = keypoints[:max_features]
        descriptors = descriptors[:max_features]
        all_features.append(descriptors)


In [76]:
print(len(all_features))
print(len(image_paths))

1491
1491


# VLAD

In [39]:
from vlad import VLAD

## VLAD use

In [40]:
vlad = VLAD(k=64, n_vocabs=1, norming="RN", lcs=True).fit(all_features)
# vlad = VLAD(k=16, n_vocabs=1, norming="RN", lcs=True).fit(all_features)

(38114310, 128)
Training vocab #1
Training KMeans...
Finding rotation-matrices...


In [56]:
# vlad_database = vlad._extract_vlads(all_features)
vlad_database = vlad.databases

In [57]:
vlad_database.shape

(1491, 8192)

# BOF

In [75]:
from bof import BOF

In [87]:
# m_bof = BOF(k=1024)
m_bof = BOF(k=20480)
m_bof.fit(all_features)

In [88]:
m_bof.databases.shape

(1491, 20480)

# Fisher Vector

In [9]:
from fisher_vector import FisherVector
fivec = FisherVector(n_components=64)
fivec.fit(all_features)

c:\Users\thanh\anaconda3\envs\ml\Lib\site-packages\sklearn\mixture\_base.py:270: ConvergenceWarning: Best performing initialization did not converge. Try different init parameters, or increase max_iter, tol, or check for degenerate data.
  warnings.warn(


In [10]:
fivec.databases.shape

(10200, 8192)

# ADC

In [10]:
from cq import CQ

(1000, 80)


In [67]:
cq = CQ(k=8)
cq.fit(vlad_database)
re_vlad_database = cq.residual(vlad_database)

In [68]:
re_vlad_database.shape, vlad_database.shape

((1491, 8192), (1491, 8192))

In [89]:
from adc import ADC

In [90]:
adc = ADC(m=32, k=1024)
# adc = ADC(m=16, k=256)

In [ ]:
adc.fit(fivec.databases)

In [91]:
adc.fit(m_bof.databases)

In [70]:
# adc.fit(vlad_database)
adc.fit(re_vlad_database)

In [92]:
adc.databases.shape

(1491, 32)

# Test 1

In [93]:
import json
with open('datasets/INRIA/groundtruth.json', 'r') as file:
    data = json.load(file)
    for key in data:
        new_similar = []
        for img in data[key]['similar']:
            for idx, path in enumerate(image_paths):
                if img in path:
                    new_similar.append(idx)
        data[key]['similar'] = new_similar

In [ ]:

outputs = []
for key in data:
    path = 'datasets/INRIA/images/' + data[key]['query']
    key_points, descriptors = extract_sift_features(path)
    # query_vlad = vlad.transform([descriptors])
    
    # re_query_vlad = cq.residual(query_vlad)
    # probabilities = adc.predict_proba(re_query_vlad)
 
    # probabilities = adc.predict_proba(query_vlad)

    query_bof = m_bof.transform([descriptors])
    re_query_bof = cq.residual(query_bof)

    probabilities = adc.predict_proba(re_query_bof)
    
    # query_fish = fivec.transform([descriptors])
    # probabilities = adc.predict_proba(query_fish)

    sorted_indices = np.argsort(probabilities)

    output_per_query = []
    for i, index in enumerate(sorted_indices):
        print(f"Original index: {index}, Probability: {probabilities[index]}")
        if i > 0:
            output_per_query.append(index)
            # if len(output_per_query) == len(data[key]['similar']):
            #     break
            if len(output_per_query) == 30:
                break
    
    outputs.append(output_per_query)

Original index: 0, Probability: 0.14782298519428716
Original index: 1, Probability: 0.18199481574599982
Original index: 15, Probability: 0.217730937629441
Original index: 1394, Probability: 0.23717003033102585
Original index: 1392, Probability: 0.23717003033102585
Original index: 1045, Probability: 0.24883334554883185
Original index: 13, Probability: 0.25747325849897273
Original index: 908, Probability: 0.27352139032385286
Original index: 909, Probability: 0.2814708456822711
Original index: 907, Probability: 0.2815374432618943
Original index: 222, Probability: 0.2844584920837284
Original index: 5, Probability: 0.2900450648715709
Original index: 1044, Probability: 0.29580704646845635
Original index: 14, Probability: 0.306484411855777
Original index: 1436, Probability: 0.3138140068849292
Original index: 326, Probability: 0.34554121948425864
Original index: 69, Probability: 0.35554444516437006
Original index: 1364, Probability: 0.3702221064404396
Original index: 341, Probability: 0.381152

In [95]:
from metrics.mAP import compute_map

In [96]:
compute_map(data, outputs)

0.31003789524421943

In [38]:
compute_map(data, outputs)

0.5263370842598865

In [34]:
compute_map(data, outputs)

0.5446192870841833

In [31]:
compute_map(data, outputs)

0.5446192870841833

In [19]:
compute_map(data, outputs)

0.4353588882737381

In [45]:
# Fisher (k = 16) ADC (m=16, k=256)
compute_map(data, outputs)

0.4323582783182517

In [ ]:
# Fisher (k = 64) ADC (m=64, k=1024)
compute_map(data, outputs)

0.43252457417325424

In [61]:
# VLAD (k=16) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.5376465998449635

In [ ]:
# VLAD (k=64) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

In [70]:
# BOF (k=1024) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.41684722833359034

In [ ]:
# BOF (k=1024) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

0.45380486841201123

In [83]:
# BOF (k=20480) ADC (m=32, k=1024) Top k : 30
compute_map(data, outputs)

0.3191882686872316

In [88]:
# BOF (k=20480) ADC (m=16, k=256) Top k : 30
compute_map(data, outputs)

0.35210481160541585

# Test 2

In [14]:
from metrics.score4 import calculate_gr_score

In [26]:
similarity_matrix = []
for path in image_paths:
    key_points, descriptors = extract_sift_features(path)
    
    # query_fish = fivec.transform([descriptors])
    # probabilities = adc.predict_proba(query_fish)
    # query_bof = m_bof.transform([descriptors])
    # probabilities = adc.predict_proba(query_bof)
    query_vlad = vlad.transform([descriptors])
    probabilities = adc.predict_proba(query_vlad)
    similarity_matrix.append(probabilities)

In [17]:
len(similarity_matrix[0]), len(similarity_matrix)

(10200, 10200)

In [28]:
calculate_gr_score(np.array(similarity_matrix))

1.7

In [57]:
calculate_gr_score(np.array(similarity_matrix))

1.6491176470588236

In [47]:
# BOF k=1024 ADC m=16 k=256
calculate_gr_score(np.array(similarity_matrix))

2.082549019607843

In [39]:
# BOF k=1024 ADC m=32 k=1024
calculate_gr_score(np.array(similarity_matrix))

2.2905882352941176

In [26]:
# Fisher k=64 ADC m=32 k=1024
calculate_gr_score(np.array(similarity_matrix))

2.0431372549019606

In [ ]:
# Fisher k=16 ADC m=16 k=256
calculate_gr_score(np.array(similarity_matrix))

2.241862745098039

# KDTree

In [ ]:
kdtree=cKDTree(vlad.database)

In [ ]:
def display_image_with_vector(image_path, vlad_vector, title):
    """
    Displays an image with its descriptor vector.

    Args:
        image_path (str): Path to the image.
        vlad_vector (numpy.ndarray): VLAD vector.
        title (str): Title for the plot.
    """
    image = cv2.imread(image_path)
    plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()


# Function to find and display similar images
def search_similar_images(query_vector, kdtree, vlad_vectors_reduced,
                          num_results=1):
    """
    Finds and displays similar images based on a query vector.

    Args:
        query_vector (numpy.ndarray): Query vector.
        kdtree: KD-Tree built on the training set.
        vlad_vectors_reduced: Reduced VLAD vectors for the training set.
        num_results (int): Number of similar images to display.
    """
    _, indices = kdtree.query(query_vector, num_results)
    print(indices)

    if num_results == 1:
        idx = int(indices[0])  # Access the first index
        image_path = image_paths[idx]
        display_image_with_vector(image_path,
                                  vlad_vectors_reduced[idx], "Similar Image")
    else:
        for i in range(num_results):
            idx = int(indices[i])
            image_path = image_paths[idx]
            display_image_with_vector(image_path,
                                      vlad_vectors_reduced[idx], f"Similar Image {i + 1}")


# Function to find the path of a similar image
def search_similar_images_path(query_vector, kdtree, vlad_vectors_reduced,
                               num_results=1):
    """
    Finds the path of a similar image based on a query vector.

    Args:
        query_vector (numpy.ndarray): Query vector.
        kdtree: KD-Tree built on the training set.
        vlad_vectors_reduced: Reduced VLAD vectors for the training set.
        num_results (int): Number of similar images to find.

    Returns:
        str: Path of the similar image.
    """
    _, indices = kdtree.query(query_vector, num_results)

    if num_results == 1:
        idx = int(indices[0])  # Access the first index
        image_path = image_paths[idx]
        return image_path
    else:
        for i in range(num_results):
            idx = int(indices[i])
            image_path = image_paths[idx]
            return image_path

In [ ]:
vlad.database.shape

In [ ]:
search_similar_images(vlad.database[0], kdtree, vlad.database, 3)